In [ ]:
import os
import time
from prometheus_client import start_http_server, Gauge, Enum
import requests
import json
import mysql.connector
import pandas as pd
from jproperties import Properties
import datetime

debug= True

configs = Properties()

if debug:
    with open('gyan-core-agent-rand.properties', 'rb') as read_prop:
        configs.load(read_prop)
else:
    with open('/opt/gyan/core/gyan-core-agent-rand-V2/gyan-core-agent-rand.properties', 'rb') as read_prop:
        configs.load(read_prop)

db_configs_dict = {}
items_view = configs.items()
for item in items_view:
    db_configs_dict[item[0]] = item[1].data


polling_interval_seconds = int(db_configs_dict["POLLING_INTERVAL_SECONDS"])

exporter_port = int(db_configs_dict["APP_PORT"])

host= str(db_configs_dict["db_host"])
db_name = str(db_configs_dict["db_name"])
password = str(db_configs_dict["db_password"])
database = str(db_configs_dict["db_database_name"])
user = str(db_configs_dict["db_user"])

client_name = str(db_configs_dict["client_name"])
column_names = db_configs_dict["column_name"].split(",")



db_connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

debug=True
#load json
if debug == True:
    json_location = str(db_configs_dict["json_location"])
else:
    json_location = str(db_configs_dict["folder_location"]) + str(db_configs_dict["json_location"])


while True:
    cursor = db_connection.cursor()

    # Get timestamp value
    current_timestamp = datetime.datetime.now()
    
    # Get Json Peak upload speed value
    f = open(json_location)
    data = json.load(f)

    total_attached_user  = data[client_name+"_1_"+column_names[0]]
    total_rejected_user = data[client_name+"_1_"+column_names[1]]
    peak_upload_speed = data[client_name+"_1_"+column_names[3]]

    peak_download_speed  = data[client_name+"_1_"+column_names[2]]
    enodeb_shutdown_count = data[client_name+"_1_"+column_names[4]]
    handoverFailureCount = data[client_name+"_1_"+column_names[5]]

    totalBearerActiveUser  = data[client_name+"_1_"+column_names[6]]
    totalBearerRejectedUser = data[client_name+"_1_"+column_names[7]]
    total_dropped_packets = data[client_name+"_1_"+column_names[8]]

    total_users  = data[client_name+"_1_"+column_names[9]]
    enodeb_connected_count = data[client_name+"_1_"+column_names[10]]
    enodeb_connection_status = data[client_name+"_1_"+column_names[11]]


    
    # SQL query and values
    MySQL_insert_query = "INSERT INTO core_stats (client_id, stats_timestamp, total_attached_user, total_rejected_user, peak_upload_speed, peak_download_speed, enodeb_shutdown_count,handover_failure_count,bearer_active_user_count,bearer_rejected_user_count,total_users,total_dropped_packets,enodeb_connected_count,enodeb_connection_status) VALUES (%s, %s, %s, %s, %s, %s, %s,%s,%s, %s, %s, %s, %s,%s)"

    the_value = (client_name, str(current_timestamp), total_attached_user, total_rejected_user, peak_upload_speed,peak_download_speed,
                enodeb_shutdown_count,   handoverFailureCount, totalBearerActiveUser, totalBearerRejectedUser, total_dropped_packets,  total_users, enodeb_connected_count,enodeb_connection_status)
    
    # Insert Command
    cursor.execute(MySQL_insert_query, the_value)
    # try:
    db_connection.commit()
    cursor.close()
    
    if debug:
        time.sleep(10)
    else:
        
        time.sleep(120)
    


In [2]:
from jproperties import Properties

configs = Properties()

with open('gyan-core-agent-rand.properties', 'rb') as read_prop:
    configs.load(read_prop)

db_configs_dict = {}
items_view = configs.items()
for item in items_view:
    db_configs_dict[item[0]] = item[1].data

json_location = str(db_configs_dict["json_location"])
f = open(json_location)
data = json.load(f)
polling_interval_seconds = int(db_configs_dict["POLLING_INTERVAL_SECONDS"])

exporter_port = int(db_configs_dict["APP_PORT"])

host= str(db_configs_dict["db_host"])
db_name = str(db_configs_dict["db_name"])
password = str(db_configs_dict["db_password"])
database = str(db_configs_dict["db_database_name"])
user = str(db_configs_dict["db_user"])

client_name = str(db_configs_dict["client_name"])
column_names = db_configs_dict["column_name"].split(",")
total_attached_user  = data[client_name+"_1_"+column_names[0]]
total_rejected_user = data[client_name+"_1_"+column_names[1]]
peak_upload_speed = data[client_name+"_1_"+column_names[3]]

peak_download_speed  = data[client_name+"_1_"+column_names[2]]
enodeb_shutdown_count = data[client_name+"_1_"+column_names[4]]
handoverFailureCount = data[client_name+"_1_"+column_names[5]]

totalBearerActiveUser  = data[client_name+"_1_"+column_names[6]]
totalBearerRejectedUser = data[client_name+"_1_"+column_names[7]]
total_dropped_packets = data[client_name+"_1_"+column_names[8]]

total_users  = data[client_name+"_1_"+column_names[9]]
enodeb_connected_count = data[client_name+"_1_"+column_names[10]]
enodeb_connection_status = data[client_name+"_1_"+column_names[11]]


In [ ]:
total_users

In [ ]:
column_names

In [ ]:
data

In [3]:
the_value = (client_name, str(current_timestamp), total_attached_user, total_rejected_user, peak_upload_speed,peak_download_speed,
            enodeb_shutdown_count,   handoverFailureCount, totalBearerActiveUser, totalBearerRejectedUser, total_dropped_packets,  total_users, enodeb_connected_count,enodeb_connection_status)


tuple